# Práctica 1: Reglas de asociación

Imports necesarios para la realización de la práctica

In [30]:
import numpy as np
import pandas as pd
import sklearn as sk
import mlxtend as mlx
import apyori as apy
import matplotlib as mpl
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [31]:
itemsetOG = pd.read_csv("BlackFriday.csv")
#itemset = itemsetOG.sample(frac=0.001)
itemset = itemsetOG.sample(20)
itemset

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
198849,1000601,P00138542,F,18-25,20,B,3,1,5,6.0,NaN
173563,1003589,P00294242,F,46-50,0,B,1,1,4,5.0,NaN
9737,1003049,P00154842,F,51-55,0,C,0,1,14,NaN,NaN
169333,1002996,P00198042,M,18-25,0,B,1,0,11,16.0,NaN
61367,1004658,P00244242,M,26-35,4,B,3,1,1,2.0,15.0
77299,1004997,P00112142,M,0-17,10,A,1,0,1,2.0,14.0
163860,1000352,P00224142,M,18-25,4,A,0,0,8,NaN,NaN
106730,1002321,P00147942,M,26-35,7,B,4+,1,1,16.0,NaN
119034,1003507,P00083742,M,26-35,0,A,3,0,2,17.0,NaN
37588,1003842,P00087342,F,36-45,16,B,4+,0,5,14.0,16.0


#### Pregunta Nº1: Prueba al menos tres configuraciones de soporte y frecuencia para cada género, edad y tipo de producto. (1 punto)


In [32]:
# Hiperparámetros
min_support_edad = [0.01, 0.05, 0.1]
min_confidence_edad = [0.7, 0.5, 0.2]

min_support_tprod = [0.01, 0.05, 0.1]
min_confidence_tprod = [0.7, 0.5, 0.2]

min_support_genero = [0.3, 0.4, 0.5]
min_confidence_genero = [0.7, 0.5, 0.2]


Configuraciones de soporte y frecuencia

- Para Edad:

In [33]:
tablaEdad = itemset[['Product_ID', 'Age']].dropna().drop_duplicates()
tablaEdad = tablaEdad.groupby('Age').agg(lambda x: list(x)).reset_index()
tablaEdad

,Age,Product_ID
0,0-17,[P00112142]
1,18-25,"[P00138542, P00198042, P00224142, P0097142, P0..."
2,26-35,"[P00244242, P00147942, P00083742, P00128142, P..."
3,36-45,"[P00087342, P00175542, P00051642]"
4,46-50,"[P00294242, P00188942]"
5,51-55,"[P00154842, P00350042]"


In [34]:
te = TransactionEncoder()
te_ary = te.fit(tablaEdad['Product_ID'].to_list()).transform(tablaEdad['Product_ID'].to_list())
df_edad = pd.DataFrame(te_ary, columns=te.columns_)
df_edad
cont = 0
for i in min_support_edad:
    print("Para min_support = " + str(i) + " y min_confidence = " + str(min_confidence_edad[cont]) + ":")
    frequent_itemsets_edad = apriori(df_edad, min_support=i, use_colnames=True, verbose=1)
    print(frequent_itemsets_edad)
    print("")

    if frequent_itemsets_edad.empty:
        print("No hay reglas de asociación que cumplan con los requisitos.")
    else:
        rules_edad = association_rules(frequent_itemsets_edad, metric="confidence", min_threshold=min_confidence_edad[cont])
        print(rules_edad)

    print("")
    cont = cont + 1


Para min_support = 0.01 y min_confidence = 0.7:
Processing 7 combinations | Sampling itemset size 765
      support                                           itemsets
0    0.166667                                        (P00010242)
1    0.166667                                        (P00037142)
2    0.166667                                        (P00051642)
3    0.166667                                        (P00083742)
4    0.166667                                        (P00087342)
..        ...                                                ...
135  0.166667  (P00037142, P00174442, P0097142, P00198042, P0...
136  0.166667  (P00128142, P00147942, P00244242, P00083742, P...
137  0.166667  (P00174442, P0097142, P00198042, P00138542, P0...
138  0.166667  (P00338542, P00010242, P00147942, P00244242, P...
139  0.166667  (P00037142, P00174442, P0097142, P00198042, P0...

[140 rows x 2 columns]

      antecedents                                        consequents  \
0     (P00083742)    

In [35]:
#pruebas con apyori (no funciona)

# frequent_itemsets_edad = list(apy.apriori(tablaEdad['Product_ID'], min_support=0.16667))
# frequent_itemsets_edad

- Para Tipo de producto 1:

In [36]:
tablaTProd1 = itemset[['Product_ID', 'Product_Category_1']].dropna().drop_duplicates()
tablaTProd1 = tablaTProd1.groupby('Product_Category_1').agg(lambda x: list(x)).reset_index()
tablaTProd1

,Product_Category_1,Product_ID
0,1,"[P00244242, P00112142, P00147942, P00037142, P..."
1,2,[P00083742]
2,4,[P00294242]
3,5,"[P00138542, P00087342, P00175542, P00174442, P..."
4,6,[P00350042]
5,8,"[P00224142, P00188942, P00051642]"
6,11,[P00198042]
7,12,[P0097142]
8,14,[P00154842]


In [45]:
te = TransactionEncoder()
te_ary = te.fit(tablaTProd1['Product_ID'].to_list()).transform(tablaTProd1['Product_ID'].to_list())
df_tprod1 = pd.DataFrame(te_ary, columns=te.columns_)
df_tprod1

cont = 0
for i in min_support_tprod:
    print("Para min_support = " + str(i) + " y min_confidence = " + str(min_confidence_tprod[cont]) + ":")
    frequent_itemsets_tprod1 = apriori(df_tprod1, min_support=i, use_colnames=True, verbose=1)
    # print(frequent_itemsets_tprod1)
    # print("")

    if frequent_itemsets_tprod1.empty:
        print("No hay reglas de asociación que cumplan con los requisitos.")
    else:
        rules_tprod1 = association_rules(frequent_itemsets_tprod1, metric="confidence", min_threshold=min_confidence_tprod[cont])
        #print(rules_tprod1)
        k3antecedents = rules_tprod1[rules_tprod1['antecedents'].apply(lambda x: len(x)) >= 2]
        print(k3antecedents)
        k3consequents = rules_tprod1[rules_tprod1['consequents'].apply(lambda x: len(x)) >= 2]
        print(k3consequents)

    print("")
    cont = cont + 1

Para min_support = 0.01 y min_confidence = 0.7:
Processing 18 combinations | Sampling itemset size 64
               antecedents  consequents  antecedent support  \
56  (P00010242, P00138542)  (P00087342)            0.111111   
57  (P00087342, P00138542)  (P00010242)            0.111111   
58  (P00010242, P00087342)  (P00138542)            0.111111   
62  (P00010242, P00087342)  (P00174442)            0.111111   
63  (P00010242, P00174442)  (P00087342)            0.111111   
64  (P00087342, P00174442)  (P00010242)            0.111111   
68  (P00175542, P00010242)  (P00087342)            0.111111   
69  (P00175542, P00087342)  (P00010242)            0.111111   
70  (P00010242, P00087342)  (P00175542)            0.111111   
74  (P00010242, P00138542)  (P00174442)            0.111111   

    consequent support   support  confidence  lift  leverage  conviction  
56            0.111111  0.111111         1.0   9.0  0.098765         inf  
57            0.111111  0.111111         1.0   9.0  0.

- Para Tipo de producto 2:

In [38]:
tablaTProd2 = itemset[['Product_ID', 'Product_Category_2']].dropna().drop_duplicates()
tablaTProd2 = tablaTProd2.groupby('Product_Category_2').agg(lambda x: list(x)).reset_index()
tablaTProd2

,Product_Category_2,Product_ID
0,2.0,"[P00244242, P00112142, P00037142]"
1,5.0,"[P00294242, P00128142]"
2,6.0,[P00138542]
3,8.0,[P00350042]
4,14.0,[P00087342]
5,16.0,"[P00198042, P00147942]"
6,17.0,[P00083742]


In [39]:
te = TransactionEncoder()
te_ary = te.fit(tablaTProd2['Product_ID'].to_list()).transform(tablaTProd2['Product_ID'].to_list())
df_tprod2 = pd.DataFrame(te_ary, columns=te.columns_)
df_tprod2

cont = 0
for i in min_support_tprod:
    print("Para min_support = " + str(i) + " y min_confidence = " + str(min_confidence_tprod[cont]) + ":")
    frequent_itemsets_tprod2 = apriori(df_tprod2, min_support=i, use_colnames=True, verbose=1)
    print(frequent_itemsets_tprod2)
    print("")
    
    if frequent_itemsets_tprod2.empty:
        print("No hay reglas de asociación que cumplan con los requisitos.")
    else:
        rules_tprod2 = association_rules(frequent_itemsets_tprod2, metric="confidence", min_threshold=min_confidence_tprod[cont])
        print(rules_tprod2)

    print("")
    cont = cont + 1


Para min_support = 0.01 y min_confidence = 0.7:
Processing 27 combinations | Sampling itemset size 32
     support                           itemsets
0   0.142857                        (P00037142)
1   0.142857                        (P00083742)
2   0.142857                        (P00087342)
3   0.142857                        (P00112142)
4   0.142857                        (P00128142)
5   0.142857                        (P00138542)
6   0.142857                        (P00147942)
7   0.142857                        (P00198042)
8   0.142857                        (P00244242)
9   0.142857                        (P00294242)
10  0.142857                        (P00350042)
11  0.142857             (P00112142, P00037142)
12  0.142857             (P00244242, P00037142)
13  0.142857             (P00244242, P00112142)
14  0.142857             (P00294242, P00128142)
15  0.142857             (P00147942, P00198042)
16  0.142857  (P00244242, P00112142, P00037142)

               antecedents       

- Para Tipo de producto 3:

In [40]:
tablaTProd3 = itemset[['Product_ID', 'Product_Category_3']].dropna().drop_duplicates()
tablaTProd3 = tablaTProd3.groupby('Product_Category_3').agg(lambda x: list(x)).reset_index()
tablaTProd3

,Product_Category_3,Product_ID
0,5.0,[P00037142]
1,14.0,[P00112142]
2,15.0,[P00244242]
3,16.0,[P00087342]
4,18.0,[P00128142]


In [41]:
te = TransactionEncoder()
te_ary = te.fit(tablaTProd3['Product_ID'].to_list()).transform(tablaTProd3['Product_ID'].to_list())
df_tprod3 = pd.DataFrame(te_ary, columns=te.columns_)
df_tprod3

cont = 0
for i in min_support_tprod:
    print("Para min_support = " + str(i) + " y min_confidence = " + str(min_confidence_tprod[cont]) + ":")
    frequent_itemsets_tprod3 = apriori(df_tprod3, min_support=i, use_colnames=True, verbose=1)
    print(frequent_itemsets_tprod3)
    print("")

    if frequent_itemsets_tprod3.empty:
        print("No hay reglas de asociación que cumplan con los requisitos.")
    else:
        rules_tprod3 = association_rules(frequent_itemsets_tprod3, metric="confidence", min_threshold=min_confidence_tprod[cont])
        print(rules_tprod3)

    print("")
    cont = cont + 1

Para min_support = 0.01 y min_confidence = 0.7:
Processing 20 combinations | Sampling itemset size 2
   support     itemsets
0      0.2  (P00037142)
1      0.2  (P00087342)
2      0.2  (P00112142)
3      0.2  (P00128142)
4      0.2  (P00244242)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []

Para min_support = 0.05 y min_confidence = 0.5:
Processing 20 combinations | Sampling itemset size 2
   support     itemsets
0      0.2  (P00037142)
1      0.2  (P00087342)
2      0.2  (P00112142)
3      0.2  (P00128142)
4      0.2  (P00244242)

Empty DataFrame
Columns: [antecedents, consequents, antecedent support, consequent support, support, confidence, lift, leverage, conviction]
Index: []

Para min_support = 0.1 y min_confidence = 0.2:
Processing 20 combinations | Sampling itemset size 2
   support     itemsets
0      0.2  (P00037142)
1      0.2  (P00087342)
2      0.2  (P00112142)
3      0

In [42]:
tablaGenero = itemset[['Product_ID', 'Gender']].dropna().drop_duplicates()
tablaGenero = tablaGenero.groupby('Gender').agg(lambda x: list(x)).reset_index()
tablaGenero

,Gender,Product_ID
0,F,"[P00138542, P00294242, P00154842, P00087342, P..."
1,M,"[P00198042, P00244242, P00112142, P00224142, P..."


In [43]:
te = TransactionEncoder()
te_ary = te.fit(tablaGenero['Product_ID'].to_list()).transform(tablaGenero['Product_ID'].to_list())
df_genero = pd.DataFrame(te_ary, columns=te.columns_)
df_genero

cont = 0
for i in min_support_genero:
    print("Para min_support = " + str(i) + " y min_confidence = " + str(min_confidence_genero[cont]) + ":")
    frequent_itemsets_genero = apriori(df_genero, min_support=i, use_colnames=True, verbose=1)
    print(frequent_itemsets_genero)
    print("")

    if frequent_itemsets_genero.empty:
        print("No hay reglas de asociación que cumplan con los requisitos.")
    else:
        rules_genero = association_rules(frequent_itemsets_genero, metric="confidence", min_threshold=min_confidence_genero[cont])
        print(rules_genero)

    print("")
    cont = cont + 1


Para min_support = 0.3 y min_confidence = 0.7:
Processing 12 combinations | Sampling itemset size 1219
      support                                           itemsets
0         0.5                                        (P00010242)
1         0.5                                        (P00037142)
2         0.5                                        (P00051642)
3         0.5                                        (P00083742)
4         0.5                                        (P00087342)
...       ...                                                ...
4345      0.5  (P00338542, P00010242, P00147942, P00037142, P...
4346      0.5  (P00338542, P00010242, P00037142, P00174442, P...
4347      0.5  (P00338542, P00010242, P00147942, P00174442, P...
4348      0.5  (P00338542, P00147942, P00037142, P00174442, P...
4349      0.5  (P00338542, P00010242, P00037142, P00147942, P...

[4350 rows x 2 columns]

        antecedents                                        consequents  \
0       (P0001024

#### Pregunta Nº2: ¿Qué diferencias hay entre usar soporte y frecuencia? Respalda la respuesta con datos (1 punto)

#### Pregunta Nº3: ¿Qué tipo reglas desaparecen según la configuración y categorías (género, edad y tipo de producto) consideradas? ¿Por qué? (1 punto)

#### Pregunta Nº4: Para una de las configuraciones, interpreta algunas de las reglas que te hayan resultado interesantes obtenidas usando la clase association_rules y un par de configuraciones cambiando la variable metric y min_threshold. Justifica los resultados. (1 punto)


#### Pregunta Nº5: Dadas las mejores configuraciones. ¿Existen reglas o patrones que se repitan? ¿Podemos generalizar de alguna manera como se comportan los clientes? Expón las conclusiones respaldadas con datos (1 punto)